In [1]:
%load_ext autoreload

In [2]:
from certified_iris_generator import CertifiedIrisRegionGenerator
import sys
import os
import time
import numpy as np
from functools import partial
import itertools
import mcubes
import visualizations_utils as viz_utils
import iris_utils #TODO remove
from iris_plant_visualizer import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display
from sandbox import rrtiris

In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import LoadModelDirectives, Parser, ProcessModelDirectives
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.all import InverseKinematics, RevoluteJoint, RationalForwardKinematics
from pydrake.geometry.optimization import IrisOptionsRationalSpace, IrisInRationalConfigurationSpace, HPolyhedron, Hyperellipsoid
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
import pydrake.symbolic as sym
from pydrake.all import MathematicalProgram, RigidTransform, RollPitchYaw
import meshcat
from pydrake.all import GenerateSeedingPolytope
import pydrake.multibody.rational_forward_kinematics as rational_forward_kinematics
from pydrake.multibody.rational_forward_kinematics import FindEpsilonLower, FindEpsilonLowerVector, FindEpsilonUpperVector
from pydrake.solvers import mathematicalprogram as mp

# Build Plant


In [4]:
q0 = [0.0, 0.0, 0.0]
q_low  = [-1.7, -2., -1.7]
q_high = [ 1.7,  2.,  1.7]


In [5]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
oneDOF_iiwa_asset = FindResourceOrThrow("drake/sos_iris_certifier/assets/oneDOF_iiwa7_with_box_collision.sdf")
twoDOF_iiwa_asset = FindResourceOrThrow("drake/sos_iris_certifier/assets/twoDOF_iiwa7_with_box_collision.sdf")

In [6]:
box_asset = FindResourceOrThrow("drake/sos_iris_certifier/assets/box_small.urdf")

models = []
models.append(parser.AddModelFromFile(box_asset))
models.append(parser.AddModelFromFile(twoDOF_iiwa_asset))
models.append(parser.AddModelFromFile(oneDOF_iiwa_asset))



locs = [[0.,0.,0.],[0.,.5,0.],[0.,-.5,0.]]
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("base", models[0]), RigidTransform(locs[0]))
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("iiwa_twoDOF_link_0", models[1]), RigidTransform(RollPitchYaw([0,0, -np.pi/2]).ToRotationMatrix(), locs[1]))
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("iiwa_oneDOF_link_0", models[2]), RigidTransform(RollPitchYaw([0,0, -np.pi/2]).ToRotationMatrix(), locs[2]))


plant.Finalize()

idx = 0
for model in models:
    for joint_index in plant.GetJointIndices(model):
        joint = plant.get_mutable_joint(joint_index)
        if isinstance(joint, RevoluteJoint):
            joint.set_default_angle(q0[idx])
            joint.set_position_limits(lower_limits= np.array([q_low[idx]]), upper_limits= np.array([q_high[idx]]))
            idx += 1
        
            
#q_low = plant.GetPositionLowerLimits().tolist()
#q_high = plant.GetPositionUpperLimits().tolist()

Ratfk = RationalForwardKinematics(plant)


In [7]:
do_viz = True
visualizer = IrisPlantVisualizer(plant, builder, scene_graph, viz_role=Role.kIllustration)
diagram = visualizer.diagram
visualizer.visualize_collision_constraint(N = 30)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6000...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
Connected to meshcat-server.


In [8]:
sliders = []
sliders.append(widgets.FloatSlider(min=q_low[0], max=q_high[0], value=0, description='q0'))
sliders.append(widgets.FloatSlider(min=q_low[1], max=q_high[1], value=0, description='q1'))
sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = q0.copy()
def handle_slider_change(change, idx):
    q[idx] = change['new']
    #print(q, end="\r")
    visualizer.showres(q)
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)

visualizer.jupyter_cell()

FloatSlider(value=0.0, description='q0', max=1.7, min=-1.7)

FloatSlider(value=0.0, description='q1', max=2.0, min=-2.0)

FloatSlider(value=0.0, description='q2', max=1.7, min=-1.7)

In [9]:
#filter fused joints self collisions 
digaram = visualizer.diagram
context = visualizer.diagram_context
plant_context = visualizer.plant_context
sg_context = scene_graph.GetMyContextFromRoot(context)
inspector = scene_graph.model_inspector()

pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()
print(len(inspector.GetCollisionCandidates()), "->", len(pairs))

gids = [gid for gid in inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)]
get_name_of_gid = lambda gid : inspector.GetName(gid)
gids.sort(key=get_name_of_gid)
iiwa_oneDOF_gids = [gid for gid in gids if "iiwa7_oneDOF::" in get_name_of_gid(gid)]
iiwa_twoDOF_gids = [gid for gid in gids if "iiwa7_twoDOF::" in get_name_of_gid(gid)]

oneDOF_fused_col_geom = iiwa_oneDOF_gids[2:]
iiwa_oneDOF_fused_set = GeometrySet(oneDOF_fused_col_geom)
twoDOF_fused_col_geom = iiwa_twoDOF_gids[4:]
iiwa_twoDOF_fused_set = GeometrySet(twoDOF_fused_col_geom)
# print([get_name_of_gid(gid) for gid in oneDOF_fused_col_geom])
# print([get_name_of_gid(gid) for gid in twoDOF_fused_col_geom])
scene_graph.collision_filter_manager()\
            .Apply(CollisionFilterDeclaration().ExcludeWithin(iiwa_oneDOF_fused_set))
scene_graph.collision_filter_manager()\
            .Apply(CollisionFilterDeclaration().ExcludeWithin(iiwa_twoDOF_fused_set))
pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()
print(len(inspector.GetCollisionCandidates()), "->", len(pairs))

150 -> 150
137 -> 150


# Setup IRIS Options and Generate Regions

In [10]:
# seed_points_q = np.array([[0.0, 0, 0], # zero config
#                         [0.0, -1.3, -1.3],  # start
#                         [0.9, -1.5, -0.8]     # goal
#                          ])
# # More interesting seed points
seed_points_q = np.array([[0.0, 0, 0], # zero config
                        [ 1.3,  2.0, -1.7],  # start: right over left
                        [ 0.9, -1.3, -1.0],  # goal: left over right
                        [-0.6, -0.9, -1.3],
                        [-0.3, -0.5, -1.7]   # passing
                         ])
# seed_points_q = np.array([[0.0, 0, 0], # zero config
#                         [0.8, 1.3, -0.8],  # START: blue low green up
#                         [0.1, 0.9, -1.2],     # GOAL: green low other up
#                         [0.2, 1.6, -0.6],
#                         [0.5, -1.9, -0.9]
#                          ])[:,(0,1,2)]   # passing


seed_points = np.array([Ratfk.ComputeTValue(seed_points_q[idx], np.zeros((3,)))\
                        for idx in range(seed_points_q.shape[0])])
if do_viz:
    visualizer.plot_seedpoints(seed_points)

start = seed_points[1,:]
goal = seed_points[2,:]

#compute limits in t-space
limits_t = []
for q in [q_low, q_high]:
    limits_t.append(Ratfk.ComputeTValue(np.array(q), np.zeros((3,)) ))
    
starting_poly = HPolyhedron.MakeBox(limits_t[0], limits_t[1])
context = diagram.CreateDefaultContext()
q_star = np.zeros(3)

In [11]:
regions = []

iris_options = IrisOptionsRationalSpace()
iris_options.require_sample_point_is_contained = True
iris_options.iteration_limit = 20
iris_options.configuration_space_margin = 1e-5
iris_options.max_faces_per_collision_pair = 60
iris_options.termination_threshold = -1
iris_options.q_star = np.zeros(3)
iris_options.relative_termination_threshold = 0.05
iris_options.enable_ibex = False
#deprecated
iris_options.certify_region_with_sos_during_generation = False
iris_options.certify_region_with_sos_after_generation = False

for i, s in enumerate(seed_points):
    plant.SetPositions(plant.GetMyMutableContextFromRoot(context), s)
    if False:
        #starting_hpolyhedron = regions[i-1]
        r = IrisInRationalConfigurationSpace (plant, plant.GetMyContextFromRoot(context),
                                              iris_options, starting_hpolyhedron)
    else:
        r = IrisInRationalConfigurationSpace(plant, plant.GetMyContextFromRoot(context), iris_options)
    regions.append(r)
    print(f'Completed region: {i+1}/{len(seed_points)}')
    print(f"Sample point contained = {np.all(r.A()@s <= r.b())}")
    print(f"Sample point contained componentwise = {r.A()@s <= r.b()}")
    print()

Completed region: 1/5
Sample point contained = True
Sample point contained componentwise = [ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True]

Completed region: 2/5
Sample point contained = True
Sample point contained componentwise = [ True  True  True  True  True  True]

Completed region: 3/5
Sample point contained = True
Sample point contained componentwise = [ True  True  True  True  True  True]

Completed region: 4/5
Sample point contained = True
Sample point contained componentwise = [ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True]

Completed region: 5/5
Sample point contained = False
Sample point contained componentwise = [ True  True  True  True  True  True  True  True False False False  True
  True  True  True  True  True  True  True  True  True  True  True]



In [12]:
if do_viz:
    visualizer.plot_regions(regions, ellipses=None, region_suffix='original')

In [13]:
regions_default = []
max_vects = []
min_vects = []
num_perm_dim = 2
num_rot = 2

for i, s in enumerate(seed_points):
    regions_default.append(GenerateSeedingPolytope(s, num_perm_dim, num_rot))
#     eps_min_vect = FindEpsilonLowerVector(regions_default[i].A(), 
#                                       regions_default[i].b(), 
#                                       limits_t[0], limits_t[1], s)
#     min_vects.append(eps_min_vect)
#     eps_max_vect = FindEpsilonUpperVector(regions_default[i].A(), 
#                                       regions_default[i].b(), 
#                                       limits_t[0], limits_t[1])
#     max_vects.append(eps_max_vect)
    
    

In [14]:
regions_default_small = []
regions_default_big = []
# for i, r in enumerate(regions_default):
#     regions_default_small.append(HPolyhedron(regions_default[i].A(), regions_default[i].b() + 0.75*min_vects[i]))
#     regions_default_big.append(HPolyhedron(regions_default[i].A(), regions_default[i].b() + max_vects[i]))
    
if do_viz:
    visualizer.plot_regions(regions_default, ellipses=None, region_suffix='_default')
#     visualizer.plot_regions(regions_default_small, ellipses=None, region_suffix='_small')
#     visualizer.plot_regions(regions_default_big, ellipses=None, region_suffix='_big')

In [13]:
cspace_free_region = rational_forward_kinematics.CspaceFreeRegion(diagram, plant, scene_graph,
                                   rational_forward_kinematics.SeparatingPlaneOrder.kAffine,
                                   rational_forward_kinematics.CspaceRegionType.kGenericPolytope)
filtered_collision_pairs = set()
solver_options = mp.SolverOptions()
solver_options.SetOption(mp.CommonSolverOption.kPrintToConsole, 1)

In [16]:
vector_bisection_search_options_list = []
scalar_bisection_search_options_list = []
bilinear_alternation_options_list = []
integrated_search_options_list = []


search_tuples = []

for i, r in enumerate(regions_default):
    vector_bisection_search_options = rational_forward_kinematics.VectorBisectionSearchOption()
    vector_bisection_search_options.max_iters = 20
    vector_bisection_search_options.max_feasible_iters = 5
    vector_bisection_search_options.search_d = True
    vector_bisection_search_options.epsilon_min = FindEpsilonLowerVector(r.A(), r.b(),
                                                             limits_t[0], limits_t[1], seed_points[i,:])
    vector_bisection_search_options.epsilon_max = 0.25*FindEpsilonUpperVector(r.A(), 
                                      r.b(), 
                                      limits_t[0], limits_t[1])
    vector_bisection_search_options_list.append(vector_bisection_search_options)
    
    scalar_bisection_search_options = rational_forward_kinematics.BinarySearchOption()
    scalar_bisection_search_options.epsilon_max = 1
    scalar_bisection_search_options.max_iters = 15
    scalar_bisection_search_options.search_d = True
    scalar_bisection_search_options.epsilon_min = FindEpsilonLower(r.A(), r.b(),
                                                         limits_t[0], limits_t[1],
                                                         seed_points[i,:])
    scalar_bisection_search_options_list.append(scalar_bisection_search_options)
    
    
    bilinear_alternation_options = rational_forward_kinematics.BilinearAlternationOption()
    bilinear_alternation_options.max_iters = 20
    bilinear_alternation_options.lagrangian_backoff_scale = 1e-6
    bilinear_alternation_options.polytope_backoff_scale = 1e-6
    bilinear_alternation_options_list.append(bilinear_alternation_options)
    
    integrated_search_options = rational_forward_kinematics.IntegratedRegionSearchOptions()
    integrated_search_options.vector_bisection_search_options = vector_bisection_search_options
    integrated_search_options.scalar_binary_search_options = scalar_bisection_search_options
    integrated_search_options.bilinear_alternation_options = bilinear_alternation_options
    integrated_search_options.max_method_switch = 2
    integrated_search_options.use_vector_bisection_search = True
    integrated_search_options_list.append(integrated_search_options)
    
    search_tuples.append((r, integrated_search_options, seed_points[i,:]))

In [26]:
def launch_integrated_search(region, integrated_search_options, seed_point):
    return cspace_free_region.IntegratedCSpacePolytopeSearch(q_star,filtered_collision_pairs,
                                                      region.A(), region.b(),
                                                     integrated_search_options, 
                                                      solver_options, seed_point
                                                     )
def launch_vector_bisection_search(region, integrated_search_options, seed_point):
    return cspace_free_region.IntegratedCSpacePolytopeSearch(q_star,filtered_collision_pairs,
                                                      region.A(), region.b(),
                                                     integrated_search_options, 
                                                      solver_options, seed_point
                                                     )

In [ ]:
ret_vals = []
for tup in search_tuples:
    ret_vals.append(launch_integrated_search(*tup))

In [14]:
i = 1
r = regions[i]
s = seed_points_q[i]
vector_bisection_search_options = rational_forward_kinematics.VectorBisectionSearchOption()
vector_bisection_search_options.max_iters = 1
vector_bisection_search_options.max_feasible_iters = 20
vector_bisection_search_options.search_d = True
vector_bisection_search_options.epsilon_min = FindEpsilonLowerVector(r.A(), r.b(),
                                                         limits_t[0], limits_t[1], seed_points[i,:])
vector_bisection_search_options.epsilon_max = 0.98*vector_bisection_search_options.epsilon_min #np.zeros(r.b().shape[0])

In [15]:
d_final = cspace_free_region.CspacePolytopeBisectionSearchVector(q_star,set(),
                                                      r.A(), r.b(),
                                                     vector_bisection_search_options, 
                                                      solver_options, seed_points_q)

[2022-02-03 21:14:14.231] [console] [info] bilinear alt on d succeeded
[2022-02-03 21:14:14.250] [console] [info] Solver time 1.4421191215515137


In [16]:
r_new = HPolyhedron(r.A(), d_final)
if do_viz:
    visualizer.plot_regions([r_new], ellipses=None, region_suffix='_new')

In [17]:
planes = cspace_free_region.separating_planes()

In [18]:
planes[0].decision_variables

RuntimeError: dtype=object arrays must be copied, and cannot be referenced

In [ ]:
#refine with CPP code:
cpp_editted_scalar_regions = []
for i in range(seed_points.shape[0]):
    s = seed_points[i,:]
    region_init = regions[i]
    region_init, region = iris_scalar_handle(s, region_init)
    cpp_editted_scalar_regions.append(region)
    print(f"Completed {i+1}/{seed_points.shape[0]}")

In [ ]:
if do_viz:
    visualizer.plot_regions(cpp_editted_regions, ellipses=None, region_suffix='cpp_editted_regions')

In [ ]:
#setup refine with python
iris_kwargs = {
    'iris_starting_ellipse_vol': 1e-5,
    'iris_plane_pullback': 1e-4,
    'iris_max_faces': -1
}
iris_generator = CertifiedIrisRegionGenerator(visualizer.diagram, plant, scene_graph, **iris_kwargs)
regions, ellipses = iris_generator.iris_in_rational_space(seed_points)
iris_generator.initalize_certifier(plane_order = 1, strict_pos_tol = 1e-4)

In [ ]:
if do_viz:
    visualizer.plot_regions(iris_generator.regions, region_suffix = '_python_snopt')

In [ ]:
# refine with python
do_linesearch_cert = True
if do_linesearch_cert:
    iris_generator.certify_and_adjust_regions_by_linesearch(1e-5)

In [ ]:
if do_viz:
    visualizer.plot_regions(iris_generator.linesearch_regions, region_suffix = '_new_by_linesearch')

In [11]:
from pydrake.all import Solve, eq, le, ge

#lam*(binner_new - pullback) - bouter<= -1e-10
def solve_pullback(b_outer, b_inner_new, lam, tol = 1e-9):
    prog = MathematicalProgram()
    pullback = prog.NewContinuousVariables(len(b_inner_new), 'pullback')
    prog.AddLinearConstraint(-lam, -np.inf*np.ones(len(b_outer)).reshape(-1,1) ,b_outer - tol - lam@b_inner_new, pullback)
    prog.AddBoundingBoxConstraint(0, np.inf, pullback)
    res = Solve(prog)
    pullback_sol = res.GetSolution(pullback)
    return res.is_success(), pullback_sol
    
def solve_hpoly_containment_problem(r_outer, r_inner):
    A_outer = r_outer.A()
    A_inner = r_inner.A()
    b_outer = r_outer.b()
    b_inner = r_inner.b()
    prog = MathematicalProgram()
    lam = prog.NewContinuousVariables(len(b_outer), len(b_inner), 'lam')
    prog.AddBoundingBoxConstraint(0.0, np.inf, lam)
    for col in range(A_outer.T.shape[1]):
        prog.AddLinearEqualityConstraint(A_inner.T, A_outer.T[:,col], lam.T[:,col])
        
    
    for r in range(lam.shape[0]):
        prog.AddLinearConstraint(b_inner[:,np.newaxis].T, np.array([-np.inf]), np.array([b_outer[r]- 1e-10]), lam[r, :])

    res =  Solve(prog)
    lam_sol = res.GetSolution(lam)
    return res.is_success(), lam_sol

def solve_volume_maximization_step(r_outer, r_inner, lam_sol, seed_point):
    A_inner = r_inner.A()
    b_inner = r_inner.b()
    b_outer = r_outer.b()
    dim = A_inner.shape[1]
    print('here')
    prog = MathematicalProgram()
    d = prog.NewContinuousVariables(dim, 'd')
    C = prog.NewSymmetricContinuousVariables(dim, 'C')
    eps = prog.NewContinuousVariables(A_inner.shape[0], 'eps')
    prog.AddMaximizeLogDeterminantCost(C)
    for idx in range(len(b_inner)):
        z = prog.NewContinuousVariables(dim + 1)
        z[0] = b_inner[idx] + eps[idx] - A_inner[idx, :]@d
        z[1:] = C @ A_inner[idx, :].T
        prog.AddLorentzConeConstraint(z)  
    prog.AddLinearConstraint(lam_sol, -np.inf*np.ones_like(b_outer), b_outer - lam_sol@b_inner, eps)
    prog.AddLinearConstraint(np.eye(len(eps)), A_inner@seed_point - b_inner, np.inf*np.ones_like(eps), eps)
    #prog.AddConstraint(le(lam_sol@(b_inner + eps) - b_outer, 0))
    #prog.AddConstraint(le(A_inner@seed_point , b_inner + eps))
    prog.AddBoundingBoxConstraint(0.0, 4.0, eps)
    res = Solve(prog)
    
    eps_sol = res.GetSolution(eps)
    print('here2')
    print(eps_sol)
    new_reg = HPolyhedron(A_inner, b_inner + eps_sol)
    return res.is_success(), eps_sol, res.get_optimal_cost()


def inner_approx_HPoly_bilinear_alternation(r_outer, r_inner_seed, seed_point, maxit = 10):
    #check if contained at start
    if r_inner_seed.ContainedInOtherHPolyhedron(r_outer):
        r_inner = r_inner_seed
        past_regions = []
        for it in range(maxit):
            print('step', it)
            #get certificate of containment
            success, lam_sol = solve_hpoly_containment_problem(r_outer, r_inner)
            print('containment step: ', success, 'lambda min', np.min(lam_sol))
            if not success:
                print('terminated early, containment lost')
                return past_regions[-2]
            #use certificate to push out faces of inner polytope
            success, eps, vol_surrogate = solve_volume_maximization_step(r_outer, r_inner_seed, lam_sol, seed_point)
            print('growth step: ', success, 'volume surrogate: ', vol_surrogate)
            print('eps', eps)
            #update polytope
            #success, pullback = solve_pullback(r_outer.b(), r_inner_seed.b() + eps, lam_sol)
            #|print('pullback: ', success, 'pullback ineq', np.max(lam_sol@(r_inner_seed.b() + eps - pullback)-r_outer.b()))
            
            r_inner = HPolyhedron(r_inner.A(), r_inner_seed.b() + eps - 1e-10) #-pullback
            print(f'max inscribed ellipse volume: {r_inner.MaximumVolumeInscribedEllipsoid().Volume()}')
            past_regions.append(r_inner)
        return r_inner
    else:
        raise ValueError("Seed HPolytope not contained in outer Hpolytope")

In [12]:
iris_options = IrisOptionsRationalSpace()
iris_options.require_sample_point_is_contained = True
iris_options.iteration_limit = 20
iris_options.configuration_space_margin = 1e-6
iris_options.max_faces_per_collision_pair = 60
iris_options.termination_threshold = -1
iris_options.q_star = np.zeros(3)
iris_options.relative_termination_threshold = 0.05
iris_options.enable_ibex = False
def iris_handle(seed, domain):
    seed_q = Ratfk.ComputeQValue(seed, np.zeros((3,)))
    plant.SetPositions(plant.GetMyMutableContextFromRoot(context), seed_q)
    region = IrisInRationalConfigurationSpace(plant, plant.GetMyContextFromRoot(context),
                                              iris_options, domain)
    return region
starting_poly = HPolyhedron.MakeBox(limits_t[0], limits_t[1])

In [13]:
compare_seed = seed_points[3]
iris_reg = iris_handle(compare_seed, starting_poly)
sym_poly = GenerateSeedingPolytope(compare_seed, 2, 2)
eps_min_vect = FindEpsilonLowerVector(sym_poly.A(),
                                      sym_poly.b(),
                                      limits_t[0], limits_t[1], compare_seed)

visualizer.plot_regions([iris_reg], region_suffix = 'bil_iris')
visualizer.plot_regions([sym_poly], region_suffix = 'bil_sym')

In [14]:
small_sym_poly = HPolyhedron(sym_poly.A(), sym_poly.b() + .9 * eps_min_vect)
visualizer.plot_regions([small_sym_poly], region_suffix = 'bil_small_sym')

In [15]:
print(eps_min_vect)
print(small_sym_poly.MaximumVolumeInscribedEllipsoid().Volume())
print(small_sym_poly.PointInSet(compare_seed))
print(small_sym_poly.ContainedInOtherHPolyhedron(iris_reg))
print(iris_reg.PointInSet(compare_seed))

[-0.09999999 -0.09999999 -0.09999999 -0.10000001 -0.10000001 -0.10000001
 -0.09999999 -0.1        -0.09999999 -0.10000001 -0.1        -0.10000001
 -0.09999999 -0.09999999 -0.1        -0.10000001 -0.10000001 -0.1
 -0.09999999 -0.10000001 -0.09999999 -0.10000001 -0.09999999 -0.10000001
 -0.09999999 -0.09999999 -0.10000001 -0.10000001 -0.10000001 -0.09999999]
4.188788207091165e-06
True
True
True


In [16]:
r_new = inner_approx_HPoly_bilinear_alternation(iris_reg, small_sym_poly, compare_seed, maxit = 40)
visualizer.plot_regions([r_new], region_suffix = 'bil_new')
print(r_new.MaximumVolumeInscribedEllipsoid().Volume())
print(r_new.PointInSet(compare_seed))
print(r_new.ContainedInOtherHPolyhedron(iris_reg))

step 0
containment step:  True lambda min -2.2359333162746964e-08
here
here2
[ 1.75605824e+00  1.73904387e+00  1.75043132e+00  1.66991999e+00
  5.91242724e-01  1.65465839e+00  3.23319745e-01  5.76472672e-02
  1.75043132e+00  7.91031487e-02  7.83315007e-01 -6.83796869e-10
  1.75605824e+00  3.13392227e-01  1.53222682e-01  1.66991999e+00
  1.65170443e+00  3.80474792e-01  1.75256116e+00  1.67532328e+00
  1.75043132e+00  1.68190770e+00  1.95418850e-01  1.65465839e+00
  1.75605824e+00  2.89977810e-01  1.70476320e+00  1.66991999e+00
  1.64826399e+00  1.73186282e-01]
growth step:  True volume surrogate:  6.014665555202648
eps [ 1.75605824e+00  1.73904387e+00  1.75043132e+00  1.66991999e+00
  5.91242724e-01  1.65465839e+00  3.23319745e-01  5.76472672e-02
  1.75043132e+00  7.91031487e-02  7.83315007e-01 -6.83796869e-10
  1.75605824e+00  3.13392227e-01  1.53222682e-01  1.66991999e+00
  1.65170443e+00  3.80474792e-01  1.75256116e+00  1.67532328e+00
  1.75043132e+00  1.68190770e+00  1.95418850e-01 

here2
[0.97621698 0.98708945 0.68857869 0.96416054 1.06220435 0.87816753
 1.00383031 0.178933   1.1885282  0.46123825 0.28037144 0.13261602
 0.97621698 1.10338902 1.17651309 0.96416054 0.25698207 0.18944854
 0.39517116 0.13492927 1.1885282  0.51344345 0.18377576 0.87816753
 0.97621698 0.65786169 0.5724471  0.96416054 0.15738348 0.29997623]
growth step:  True volume surrogate:  3.44698563734316
eps [0.97621698 0.98708945 0.68857869 0.96416054 1.06220435 0.87816753
 1.00383031 0.178933   1.1885282  0.46123825 0.28037144 0.13261602
 0.97621698 1.10338902 1.17651309 0.96416054 0.25698207 0.18944854
 0.39517116 0.13492927 1.1885282  0.51344345 0.18377576 0.87816753
 0.97621698 0.65786169 0.5724471  0.96416054 0.15738348 0.29997623]
max inscribed ellipse volume: 0.13337724635295295
step 10
containment step:  True lambda min -9.682124224508618e-10
here
here2
[0.94541584 0.94968859 0.72995071 0.91048533 1.03042359 0.75356761
 0.4247875  0.18071195 1.17355505 0.4979978  0.28124653 0.13303818
 0

containment step:  True lambda min -5.551115123125783e-17
here
here2
[0.97127897 0.46821704 0.85409712 0.9178439  1.06189583 0.63691771
 0.51742296 0.19017745 1.27778349 0.5391293  0.28136309 0.14555617
 0.97127897 0.79714013 0.7748469  0.9178439  0.90745365 0.20997611
 0.5117516  0.12384468 0.85512676 0.59854311 0.18454986 0.83453021
 0.97127897 0.84093195 0.63995956 0.9178439  0.17689662 0.94018061]
growth step:  True volume surrogate:  3.1924022888342503
eps [0.97127897 0.46821704 0.85409712 0.9178439  1.06189583 0.63691771
 0.51742296 0.19017745 1.27778349 0.5391293  0.28136309 0.14555617
 0.97127897 0.79714013 0.7748469  0.9178439  0.90745365 0.20997611
 0.5117516  0.12384468 0.85512676 0.59854311 0.18454986 0.83453021
 0.97127897 0.84093195 0.63995956 0.9178439  0.17689662 0.94018061]
max inscribed ellipse volume: 0.17204649327636215
step 21
containment step:  True lambda min -5.551115123125783e-17
here
here2
[0.98014663 1.01491744 0.85487753 0.88053531 1.07332842 0.85222759
 0.5

here2
[0.97478188 0.48628754 0.85964249 0.93622449 0.55880109 0.80131519
 0.51696137 0.19691538 1.28708072 0.5625453  0.28102488 0.14686098
 0.97478188 0.74670141 0.77790593 0.93622449 0.29001878 0.22696229
 0.5295622  0.11746186 1.29018686 0.58146606 0.18500125 0.8478808
 0.97478188 0.84386574 1.12722048 0.93622449 0.17267194 0.96449263]
growth step:  True volume surrogate:  3.1899461528431714
eps [0.97478188 0.48628754 0.85964249 0.93622449 0.55880109 0.80131519
 0.51696137 0.19691538 1.28708072 0.5625453  0.28102488 0.14686098
 0.97478188 0.74670141 0.77790593 0.93622449 0.29001878 0.22696229
 0.5295622  0.11746186 1.29018686 0.58146606 0.18500125 0.8478808
 0.97478188 0.84386574 1.12722048 0.93622449 0.17267194 0.96449263]
max inscribed ellipse volume: 0.17246957881665378
step 31
containment step:  True lambda min -9.621449109906335e-07
here
here2
[0.97196357 0.48420439 0.85971171 0.9303687  0.56120453 0.84158326
 0.73587898 0.1969915  1.28431894 0.53336963 0.28102094 0.14687172
 0

In [17]:
#certify found region
cspace_free_region = rational_forward_kinematics.CspaceFreeRegion(diagram, plant, scene_graph,
                                   rational_forward_kinematics.SeparatingPlaneOrder.kAffine,
                                   rational_forward_kinematics.CspaceRegionType.kGenericPolytope)

vector_bisection_search_options = rational_forward_kinematics.VectorBisectionSearchOption()
vector_bisection_search_options.max_iters = 10
vector_bisection_search_options.max_feasible_iters = 3
vector_bisection_search_options.search_d = True
filtered_collision_pairs = set()
vector_bisection_search_options.epsilon_min = FindEpsilonLowerVector(r_new.A(), r_new.b(),
                                                         limits_t[0], limits_t[1], compare_seed)

vector_bisection_search_options.epsilon_max = 0.25*FindEpsilonUpperVector(r_new.A(), 
                                      r_new.b(), 
                                      limits_t[0], limits_t[1],)
solver_options = mp.SolverOptions()
solver_options.SetOption(mp.CommonSolverOption.kPrintToConsole, 1)

b_feasible = cspace_free_region.CspacePolytopeBisectionSearchVector(
                 q_star, filtered_collision_pairs, r_new.A(), r_new.b(),
                 vector_bisection_search_options, solver_options)
r0_grown_vector = HPolyhedron(r_new.A(), b_feasible) 

[2022-02-05 11:13:52.845] [console] [info] Solver time 4.254333972930908
[2022-02-05 11:13:59.231] [console] [info] Solver time 1.7972090244293213
[2022-02-05 11:14:09.220] [console] [info] Solver time 5.422446966171265
[2022-02-05 11:14:09.267] [console] [info] total iter=1, feasible iter = 0
[2022-02-05 11:14:21.887] [console] [info] bilinear alt on d succeeded
[2022-02-05 11:14:21.913] [console] [info] Solver time 2.0156021118164062
[2022-02-05 11:14:21.969] [console] [info] total iter=2, feasible iter = 1
[2022-02-05 11:14:34.759] [console] [info] bilinear alt on d succeeded
[2022-02-05 11:14:34.785] [console] [info] Solver time 2.195435047149658
[2022-02-05 11:14:34.844] [console] [info] total iter=3, feasible iter = 2
[2022-02-05 11:14:42.138] [console] [info] Solver time 2.877234935760498
[2022-02-05 11:14:42.181] [console] [info] total iter=4, feasible iter = 2
[2022-02-05 11:14:55.494] [console] [info] bilinear alt on d succeeded
[2022-02-05 11:14:55.520] [console] [info] Solv

In [18]:
visualizer.plot_regions([r0_grown_vector], region_suffix = 'bil_cert')

In [67]:
planes = cspace_free_region.separating_planes()
geom_touple_to_plane_dict = {}
for plane in planes:
    geom_touple_to_plane_dict[(plane.geometryA, plane.geometryB)] = (plane.a(), plane.b)

In [68]:
from pydrake.all import Evaluate

In [49]:
plane.b.Evaluate(dict(zip(plane.b.GetVariables(), q)))


RuntimeError: The following environment does not have an entry for the variable b_constant136
t[2] -> 1.7
t[1] -> 2
t[0] -> 1.7



In [48]:
q

[1.7, 2.0, 1.7]

In [69]:
b = plane.b

In [44]:
t_exp_vars = []
for idx, var in enumerate(b.GetVariables()):
    t_exp_vars.append(var)
    if idx>1:
        break

In [70]:
variables = [v for v in b.GetVariables()]

In [71]:
variables

[Variable('t[0]', Continuous),
 Variable('t[1]', Continuous),
 Variable('t[2]', Continuous),
 Variable('b_coeff136(0)', Continuous),
 Variable('b_coeff136(1)', Continuous),
 Variable('b_coeff136(2)', Continuous),
 Variable('b_constant136', Continuous)]

In [63]:
plane.b

<Expression "(b_constant136 + (t[0] * b_coeff136(0)) + (t[1] * b_coeff136(1)) + (t[2] * b_coeff136(2)))">

In [65]:
plane.b.Evaluate(dict(zip(plane.b.GetVariables(), q+q+[1])))

10.78

In [31]:
b.Evaluate({'t': np.zeros(3)})

TypeError: Evaluate(): incompatible function arguments. The following argument types are supported:
    1. (self: pydrake.symbolic.Expression, env: Dict[pydrake.symbolic.Variable, float] = {}, generator: pydrake.common._module_py.RandomGenerator = None) -> float
    2. (self: pydrake.symbolic.Expression, generator: pydrake.common._module_py.RandomGenerator) -> float

Invoked with: <Expression "(b_constant136 + (t[0] * b_coeff136(0)) + (t[1] * b_coeff136(1)) + (t[2] * b_coeff136(2)))">, {'t': array([0., 0., 0.])}

In [43]:
t_exp_vars

[Variable('t[0]', Continuous),
 Variable('t[1]', Continuous),
 Variable('t[2]', Continuous)]

In [25]:
print( geom_touple_to_plane_dict.keys())

dict_keys([(<GeometryId value=23>, <GeometryId value=62>), (<GeometryId value=23>, <GeometryId value=68>), (<GeometryId value=23>, <GeometryId value=74>), (<GeometryId value=23>, <GeometryId value=80>), (<GeometryId value=23>, <GeometryId value=86>), (<GeometryId value=23>, <GeometryId value=92>), (<GeometryId value=23>, <GeometryId value=110>), (<GeometryId value=23>, <GeometryId value=116>), (<GeometryId value=23>, <GeometryId value=122>), (<GeometryId value=23>, <GeometryId value=128>), (<GeometryId value=23>, <GeometryId value=134>), (<GeometryId value=23>, <GeometryId value=140>), (<GeometryId value=30>, <GeometryId value=62>), (<GeometryId value=30>, <GeometryId value=68>), (<GeometryId value=30>, <GeometryId value=74>), (<GeometryId value=30>, <GeometryId value=80>), (<GeometryId value=30>, <GeometryId value=86>), (<GeometryId value=30>, <GeometryId value=92>), (<GeometryId value=30>, <GeometryId value=110>), (<GeometryId value=30>, <GeometryId value=116>), (<GeometryId value=30

In [86]:
visualizer.region_to_collision_pair_to_plane_dictionary = {r0_grown_vector: geom_touple_to_plane_dict}
sliders = []
sliders.append(widgets.FloatSlider(min=q_low[0], max=q_high[0], value=0, description='q0'))
sliders.append(widgets.FloatSlider(min=q_low[1], max=q_high[1], value=0, description='q1'))
sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = q0.copy()
def handle_slider_change(change, idx):
    q[idx] = change['new']
    #print(q, end="\r")
    visualizer.show_res_with_planes(q)
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)

visualizer.jupyter_cell()

FloatSlider(value=0.0, description='q0', max=1.7, min=-1.7)

FloatSlider(value=0.0, description='q1', max=2.0, min=-2.0)

FloatSlider(value=0.0, description='q2', max=1.7, min=-1.7)

AttributeError: 'pydrake.symbolic.Expression' object has no attribute 'indeterminates'

In [93]:
a,b = geom_touple_to_plane_dict[next(iter(geom_touple_to_plane_dict.keys()))]


<Expression "(b_constant0 + (t[0] * b_coeff0(0)) + (t[1] * b_coeff0(1)) + (t[2] * b_coeff0(2)))">

In [95]:
idx = np.argsort(geom_pairs_A)

In [50]:
np.array(geom_pairs)[idx].tolist()

[[<GeometryId value=23>, <GeometryId value=68>],
 [<GeometryId value=23>, <GeometryId value=74>],
 [<GeometryId value=23>, <GeometryId value=92>],
 [<GeometryId value=23>, <GeometryId value=128>],
 [<GeometryId value=23>, <GeometryId value=110>],
 [<GeometryId value=23>, <GeometryId value=122>],
 [<GeometryId value=23>, <GeometryId value=134>],
 [<GeometryId value=23>, <GeometryId value=80>],
 [<GeometryId value=23>, <GeometryId value=62>],
 [<GeometryId value=23>, <GeometryId value=86>],
 [<GeometryId value=23>, <GeometryId value=140>],
 [<GeometryId value=23>, <GeometryId value=116>],
 [<GeometryId value=30>, <GeometryId value=68>],
 [<GeometryId value=30>, <GeometryId value=122>],
 [<GeometryId value=30>, <GeometryId value=116>],
 [<GeometryId value=30>, <GeometryId value=92>],
 [<GeometryId value=30>, <GeometryId value=86>],
 [<GeometryId value=30>, <GeometryId value=128>],
 [<GeometryId value=30>, <GeometryId value=140>],
 [<GeometryId value=30>, <GeometryId value=62>],
 [<Geometr

In [ ]:
a = cspace_free_region.separating_planes()[0]

In [ ]:
for r_new in regions_new:
    print(f'max inscribed ellipse volume: {r_new.MaximumVolumeInscribedEllipsoid().Volume()}')
    print(r_new.PointInSet(compare_seed))
    print('contained', r_new.ContainedInOtherHPolyhedron(iris_reg))
visualizer.plot_regions(regions_new, region_suffix = 'bil_regions_new4')


In [ ]:
visualizer.plot_regions([regions_new[-1]], region_suffix = 'end')

In [ ]:
iris_reg.A().shape

In [56]:
{set((plane.geometryA, plane.geometryB)) : plane for plane in planes}

TypeError: unhashable type: 'set'

In [ ]:
planes = cspace_free_region.separating_planes()

In [84]:

    #print(f'({plane.geometryA}, {plane.geometryB}')

(<GeometryId value=23>, <GeometryId value=62>
(<GeometryId value=23>, <GeometryId value=68>
(<GeometryId value=23>, <GeometryId value=74>
(<GeometryId value=23>, <GeometryId value=80>
(<GeometryId value=23>, <GeometryId value=86>
(<GeometryId value=23>, <GeometryId value=92>
(<GeometryId value=23>, <GeometryId value=110>
(<GeometryId value=23>, <GeometryId value=116>
(<GeometryId value=23>, <GeometryId value=122>
(<GeometryId value=23>, <GeometryId value=128>
(<GeometryId value=23>, <GeometryId value=134>
(<GeometryId value=23>, <GeometryId value=140>
(<GeometryId value=30>, <GeometryId value=62>
(<GeometryId value=30>, <GeometryId value=68>
(<GeometryId value=30>, <GeometryId value=74>
(<GeometryId value=30>, <GeometryId value=80>
(<GeometryId value=30>, <GeometryId value=86>
(<GeometryId value=30>, <GeometryId value=92>
(<GeometryId value=30>, <GeometryId value=110>
(<GeometryId value=30>, <GeometryId value=116>
(<GeometryId value=30>, <GeometryId value=122>
(<GeometryId value=30>, <

In [85]:
visualizer.

{(<GeometryId value=23>,
  <GeometryId value=62>): (array([<Expression "(a_constant0(0) + (t[0] * a_coeff0(0, 0)) + (t[1] * a_coeff0(0, 1)) + (t[2] * a_coeff0(0, 2)))">,
         <Expression "(a_constant0(1) + (t[0] * a_coeff0(1, 0)) + (t[1] * a_coeff0(1, 1)) + (t[2] * a_coeff0(1, 2)))">,
         <Expression "(a_constant0(2) + (t[0] * a_coeff0(2, 0)) + (t[1] * a_coeff0(2, 1)) + (t[2] * a_coeff0(2, 2)))">],
        dtype=object), <Expression "(b_constant0 + (t[0] * b_coeff0(0)) + (t[1] * b_coeff0(1)) + (t[2] * b_coeff0(2)))">),
 (<GeometryId value=23>,
  <GeometryId value=68>): (array([<Expression "(a_constant1(0) + (t[0] * a_coeff1(0, 0)) + (t[1] * a_coeff1(0, 1)) + (t[2] * a_coeff1(0, 2)))">,
         <Expression "(a_constant1(1) + (t[0] * a_coeff1(1, 0)) + (t[1] * a_coeff1(1, 1)) + (t[2] * a_coeff1(1, 2)))">,
         <Expression "(a_constant1(2) + (t[0] * a_coeff1(2, 0)) + (t[1] * a_coeff1(2, 1)) + (t[2] * a_coeff1(2, 2)))">],
        dtype=object), <Expression "(b_constant1 + (t[0

In [72]:

geom_pairs_inspector = []
for geomA, geomB in inspector.GetCollisionCandidates():
    geom_pairs_inspector.append((geomA, geomB))

        

In [74]:
inspector.GetCollisionCandidates()

{(<GeometryId value=23>, <GeometryId value=62>),
 (<GeometryId value=23>, <GeometryId value=68>),
 (<GeometryId value=23>, <GeometryId value=74>),
 (<GeometryId value=23>, <GeometryId value=80>),
 (<GeometryId value=23>, <GeometryId value=86>),
 (<GeometryId value=23>, <GeometryId value=92>),
 (<GeometryId value=23>, <GeometryId value=110>),
 (<GeometryId value=23>, <GeometryId value=116>),
 (<GeometryId value=23>, <GeometryId value=122>),
 (<GeometryId value=23>, <GeometryId value=128>),
 (<GeometryId value=23>, <GeometryId value=134>),
 (<GeometryId value=23>, <GeometryId value=140>),
 (<GeometryId value=30>, <GeometryId value=62>),
 (<GeometryId value=30>, <GeometryId value=68>),
 (<GeometryId value=30>, <GeometryId value=74>),
 (<GeometryId value=30>, <GeometryId value=80>),
 (<GeometryId value=30>, <GeometryId value=86>),
 (<GeometryId value=30>, <GeometryId value=92>),
 (<GeometryId value=30>, <GeometryId value=110>),
 (<GeometryId value=30>, <GeometryId value=116>),
 (<GeometryI

In [69]:
type(geom_pairs_inspector)

list